In [13]:
import util

import collections, itertools, pathlib, string

In [14]:
allowed_targets = util.parse_word_list(pathlib.Path('./allowed_targets.txt'),
                                       shuffle=False)
len(allowed_targets)

2309

In [15]:
collections.Counter(itertools.chain(*map(set, allowed_targets))).most_common()

[('e', 1053),
 ('a', 906),
 ('r', 835),
 ('o', 672),
 ('t', 667),
 ('i', 646),
 ('l', 645),
 ('s', 617),
 ('n', 548),
 ('u', 456),
 ('c', 446),
 ('y', 416),
 ('h', 377),
 ('d', 370),
 ('p', 345),
 ('g', 299),
 ('m', 298),
 ('b', 266),
 ('f', 206),
 ('k', 202),
 ('w', 193),
 ('v', 148),
 ('x', 37),
 ('z', 35),
 ('q', 29),
 ('j', 27)]

In [16]:
TARGET_LENGTH = len(allowed_targets[0])
for i in range(TARGET_LENGTH):
    print(f'Index {i}:', collections.Counter(w[i] for w in allowed_targets).most_common())

Index 0: [('s', 365), ('c', 198), ('b', 173), ('t', 149), ('p', 141), ('a', 140), ('f', 135), ('g', 115), ('d', 111), ('m', 107), ('r', 105), ('l', 87), ('w', 82), ('e', 72), ('h', 69), ('v', 43), ('o', 41), ('n', 37), ('i', 34), ('u', 33), ('q', 23), ('k', 20), ('j', 20), ('y', 6), ('z', 3)]
Index 1: [('a', 304), ('o', 279), ('r', 267), ('e', 241), ('i', 201), ('l', 200), ('u', 185), ('h', 144), ('n', 87), ('t', 77), ('p', 61), ('w', 44), ('c', 40), ('m', 38), ('y', 22), ('d', 20), ('b', 16), ('s', 16), ('v', 15), ('x', 14), ('g', 11), ('k', 10), ('f', 8), ('q', 5), ('z', 2), ('j', 2)]
Index 2: [('a', 306), ('i', 266), ('o', 243), ('e', 177), ('u', 165), ('r', 163), ('n', 137), ('l', 112), ('t', 111), ('s', 80), ('d', 75), ('g', 67), ('m', 61), ('p', 57), ('c', 56), ('b', 56), ('v', 49), ('y', 29), ('w', 26), ('f', 25), ('k', 12), ('x', 12), ('z', 11), ('h', 9), ('j', 3), ('q', 1)]
Index 3: [('e', 318), ('n', 182), ('s', 171), ('l', 162), ('a', 162), ('i', 158), ('c', 150), ('r', 150)

In [17]:
{
    letter: [sum(w[i] == letter for w in allowed_targets)
             for i in range(TARGET_LENGTH)]
    for letter in string.ascii_lowercase
}

{'a': [140, 304, 306, 162, 63],
 'b': [173, 16, 56, 24, 11],
 'c': [198, 40, 56, 150, 31],
 'd': [111, 20, 75, 69, 118],
 'e': [72, 241, 177, 318, 422],
 'f': [135, 8, 25, 35, 26],
 'g': [115, 11, 67, 76, 41],
 'h': [69, 144, 9, 28, 137],
 'i': [34, 201, 266, 158, 11],
 'j': [20, 2, 3, 2, 0],
 'k': [20, 10, 12, 55, 113],
 'l': [87, 200, 112, 162, 155],
 'm': [107, 38, 61, 68, 42],
 'n': [37, 87, 137, 182, 130],
 'o': [41, 279, 243, 132, 58],
 'p': [141, 61, 57, 50, 56],
 'q': [23, 5, 1, 0, 0],
 'r': [105, 267, 163, 150, 212],
 's': [365, 16, 80, 171, 36],
 't': [149, 77, 111, 139, 253],
 'u': [33, 185, 165, 82, 1],
 'v': [43, 15, 49, 45, 0],
 'w': [82, 44, 26, 25, 17],
 'x': [0, 14, 12, 3, 8],
 'y': [6, 22, 29, 3, 364],
 'z': [3, 2, 11, 20, 4]}